# Carrier Agreements & Guides (Unstructured)
Parse PDFs with Document Intelligence and produce structured tables.

In [ ]:
import os, json
from datetime import datetime
# Placeholder for using Azure Document Intelligence SDK.
# In Fabric, install package 'azure-ai-formrecognizer' or use built-in cognitive integration.
# This example emits normalized JSON ready for downstream tables.
agreements_dir = 'landing/carrier_docs'
rows = []
for f in os.listdir(agreements_dir):
    if not f.lower().endswith('.json'):
        continue
    doc = json.load(open(os.path.join(agreements_dir, f)))
    carrier = doc.get('carrier')
    eff = doc.get('effectiveDate')
    for clause in doc.get('clauses', []):
        rows.append({
            'carrier': carrier,
            'effectiveDate': eff,
            'clauseId': clause.get('id'),
            'text': clause.get('text'),
        })
import pandas as pd
df = pd.DataFrame(rows)
df.to_parquet('curated/ContractClauses.parquet', index=False)
print('Wrote clauses table at', datetime.utcnow(), 'UTC')
